In [ ]:
import os
import sys
import numpy as np
import pandas as pd

src_path = os.path.abspath(os.path.join(os.path.curdir, "src"))
sys.path.append(src_path)

from src.dataset import read_dataset, download_dataset
from src.helper import compute_dataset_statistics

In [ ]:
download_dataset

df = read_dataset()

In [ ]:
user_artists_df = df.groupby(['user_id', 'artist_id']).agg(
    timestamp = pd.NamedAgg(column='timestamp', aggfunc='max'),
).reset_index()
user_track_df = df.groupby(['user_id', 'track_id']).agg(
    timestamp = pd.NamedAgg(column='timestamp', aggfunc='max'),
).reset_index()

user_artists_df = user_artists_df.rename(columns={'artist_id': 'item_id'})
user_track_df = user_track_df.rename(columns={'track_id': 'item_id'})

user_item_df = pd.concat([user_artists_df, user_track_df])

user_item_df.info()

In [ ]:
stats = compute_dataset_statistics(user_item_df)

In [ ]:
from src.light_gcn import LightGCN

li_gcnn = LightGCN(
    df=user_item_df,
    batch_size=1024,
    embedding_size=64,
    num_layers=3,
    normalize=True,
    learning_rate=0.001,
    regularization=1e-5
)
preds = li_gcnn.fit(epochs=5, val_epoch=1)